In [1]:
# Older than Spark 2.2.0
from pyspark.mllib.fpm import FPGrowth

In [2]:
data = ["r z h k p","z y x w v u t s", "s x o n r", "x z y m t s q e", "z", "x z y r q t p"]
transactions = sc.parallelize(data).map(lambda line: line.strip().split(' '))
print type(transactions)
transactions.take(3)

<class 'pyspark.rdd.PipelinedRDD'>


[['r', 'z', 'h', 'k', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['s', 'x', 'o', 'n', 'r']]

In [3]:
model = FPGrowth.train(transactions, minSupport=0.2, numPartitions=10)

In [16]:
result = model.freqItemsets().cache()
print type(result)
result_print = result.collect()
for fi in result_print[:4]:
    print(fi)

<class 'pyspark.rdd.PipelinedRDD'>
FreqItemset(items=[u'z'], freq=5)
FreqItemset(items=[u'x'], freq=4)
FreqItemset(items=[u'x', u'z'], freq=3)
FreqItemset(items=[u'y'], freq=3)


In [5]:
MEM_AND_DISK = StorageLevel(True, True, False, True, 1)
tmp = result.map(lambda x: (map(str,x.items), x.freq)).persist(MEM_AND_DISK)

In [6]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [7]:
list(powerset([1,2,3]))

[(), (1,), (2,), (3,), (1, 2), (1, 3), (2, 3), (1, 2, 3)]

In [8]:
print tmp.map(lambda x: x[0]).take(5)
print tmp.map(lambda x: x[0]).map(lambda x: list(powerset(x))).take(5)
print tmp.map(lambda x: x[0]).flatMap(lambda x: list(powerset(x))).take(5)
print tmp.map(lambda x: x[0]).flatMap(lambda x: list(powerset(x))).map(lambda x: (x, 1)).take(5)
# print tmp.map(lambda x: x[0]).flatMap(lambda x: list(powerset(x))).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y).take(5)

[['z'], ['x'], ['x', 'z'], ['y'], ['y', 'x']]
[[(), ('z',)], [(), ('x',)], [(), ('x',), ('z',), ('x', 'z')], [(), ('y',)], [(), ('y',), ('x',), ('y', 'x')]]
[(), ('z',), (), ('x',), ()]
[((), 1), (('z',), 1), ((), 1), (('x',), 1), ((), 1)]


In [45]:
print tmp.take(5)

[(['z'], 5), (['x'], 4), (['x', 'z'], 3), (['y'], 3), (['y', 'x'], 3)]


In [43]:
tmp2 = tmp.map(lambda x: x[0])\
          .filter(lambda x: len(x) > 1)\
          .map(lambda x: list(powerset(x)))\
          .zipWithIndex()\
          .map(lambda (x, y): (y,x))\
          .flatMapValues(lambda x: x)\
          .map(lambda (x, y): (str(list(y)),x))\
        
print tmp2.take(4)

[('[]', 0), ("['x']", 0), ("['z']", 0), ("['x', 'z']", 0)]


In [10]:
result.map(lambda x: (map(str,x.items), x.freq)).flatMap(lambda x: x[0]+x[0]).take(3)

['z', 'z', 'x']

In [11]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def subsets(s):
    return map(set, powerset(s))

In [12]:
def generate_associationrule(freq_itemset):
    
    return subsets(freq_itemset[0])

In [13]:
print result.map(generate_associationrule).take(5)

[[set([]), set([u'z'])], [set([]), set([u'x'])], [set([]), set([u'x']), set([u'z']), set([u'x', u'z'])], [set([]), set([u'y'])], [set([]), set([u'y']), set([u'x']), set([u'y', u'x'])]]


##### Note:

In [14]:
print model.freqItemsets().sortBy(lambda x: x[1]).take(2)

[FreqItemset(items=[u'r', u'x'], freq=2), FreqItemset(items=[u'r', u'z'], freq=2)]


In [15]:
print model.freqItemsets().map(lambda x: x[1]).distinct().collect()

[2, 3, 4, 5]
